In [2]:
import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

import numpy as np
import pandas as pd
import json
import os

In [3]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQE\NegPenalty\VaryPen")
potential = 'AHO'
cutoff = 8

dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    tolerance.append(optimzer['tolerance'])

    success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_VQE_time'])

    

In [4]:
df = pd.DataFrame({
    'folder': folders,
    #'num_vqe': num_vqe,
    #'shots': shots,
    'tolerance': tolerance,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    #'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [5]:
df[['Delta min E', 'Delta median E']] = df[['Delta min E', 'Delta median E']].astype(float)
df = df.sort_values('Delta median E', ascending=False)
df.to_excel(os.path.join(folder_path, "table.xlsx"), index=False)
df

,folder,tolerance,Converged Runs,Mean Iter,Delta min E,Delta median E,Exact,Time
3,4,1.000000e-08,50/100,59,0.223588,0.153872,3.20101e-02,0:01:06.814434
2,3,1.000000e-08,50/100,59,0.131392,0.135755,3.20101e-02,0:01:05.055883
0,1,1.000000e-08,50/100,60,0.031877,0.124985,3.20101e-02,0:01:06.785579
1,2,1.000000e-08,50/100,60,0.048674,0.047407,3.20101e-02,0:01:07.079763
4,5,1.000000e-08,50/100,51,0.548672,0.024293,3.20101e-02,0:00:35.406851
